In [2]:
import threading
import numpy as np
import tensorflow as tf
import sys
from tkinter import Tk, Canvas, Button, Label
from PIL import ImageGrab, Image, ImageOps
from tensorflow.keras.models import load_model

print("Python Version:", sys.version)

class DrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Handwritten Digit Recognition")

        self.canvas = Canvas(root, width=400, height=400, bg="white")
        self.canvas.pack()

        try:
            self.model = load_model("full.h5")
            print("✅ Model Loaded Successfully (full.h5)")
        except Exception as e:
            print(f"⚠️ Error: Model file 'full.h5' not found! ({e})")
            self.model = None

        self.canvas.bind("<B1-Motion>", self.draw_lines)

        recognize_button = Button(root, text="Recognize", command=self.recognize_action)
        recognize_button.pack(side="left", padx=10, pady=10)

        clear_button = Button(root, text="Clear", command=self.clear_canvas)
        clear_button.pack(side="right", padx=10, pady=10)

        self.result_label = Label(root, text="Prediction: None", font=("Arial", 14))
        self.result_label.pack(pady=10)

    def draw_lines(self, event):
        x, y = event.x, event.y
        radius = 10
        self.canvas.create_oval(x-radius, y-radius, x+radius, y+radius, fill="black", outline="black")

    def clear_canvas(self):
        self.canvas.delete("all")
        self.result_label.config(text="Prediction: None")

    def recognize_action(self):
        threading.Thread(target=self._recognize).start()

    def _recognize(self):
        self.root.update()
        x = self.canvas.winfo_rootx()
        y = self.canvas.winfo_rooty()
        w = x + self.canvas.winfo_width()
        h = y + self.canvas.winfo_height()
        
        img = ImageGrab.grab(bbox=(x, y, w, h))

        img = img.convert("L")  # Convert to grayscale
        img = ImageOps.invert(img)  # Invert colors

        # Adaptive thresholding (binarization)
        img = np.array(img)
        img = np.where(img > 100, 255, 0).astype(np.uint8)

        # Find bounding box of the digit
        coords = np.column_stack(np.where(img > 0))
        if coords.shape[0] > 0:
            x_min, y_min = coords.min(axis=0)
            x_max, y_max = coords.max(axis=0)
            img = img[x_min:x_max+1, y_min:y_max+1]  # Crop to bounding box

        # Resize to 28x28
        img = Image.fromarray(img)
        img = img.resize((28, 28))

        # Convert to array and normalize
        img_array = np.array(img) / 255.0
        img_array = img_array.reshape(1, 28, 28, 1)

        if self.model:
            prediction = self.model.predict(img_array)
            confidence = np.max(prediction) * 100
            digit = np.argmax(prediction)

            if confidence < 70:
                self.result_label.config(text="Prediction: Not Recognized")
                print("⚠️ Low confidence! Not Recognized")
            else:
                self.result_label.config(text=f"Prediction: {digit} (Confidence: {confidence:.2f}%)")
                print(f"📝 Recognized Digit: {digit} (Confidence: {confidence:.2f}%)")
        else:
            print("⚠️ No model loaded!")

if __name__ == "__main__":
    root = Tk()
    app = DrawingApp(root)
    root.mainloop()


Python Version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]


✅ Model Loaded Successfully (full.h5)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step
📝 Recognized Digit: 5 (Confidence: 100.00%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
📝 Recognized Digit: 5 (Confidence: 99.42%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
📝 Recognized Digit: 6 (Confidence: 98.24%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
📝 Recognized Digit: 7 (Confidence: 95.93%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
📝 Recognized Digit: 5 (Confidence: 99.99%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
📝 Recognized Digit: 5 (Confidence: 100.00%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
📝 Recognized Digit: 5 (Confidence: 100.00%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
📝 Recognized Digit: 5 (Confidence: 100.00%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
📝 Recognized Digit: 2 (Confidence: 95.11%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
📝 Recognized Digit: 5 (Confidence: 98.87%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
⚠️ Low confidence! Not Recognized
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
📝 Recognized Digit: 4 (Con